In [1]:
import taichi as ti

[Taichi] mode=release
[Taichi] version 0.6.9, supported archs: [cpu, cuda, opengl], commit afd6650c, python 3.6.2


In [2]:

ti.init()
a = ti.var(dt=ti.f32, shape=(42,63))
a[3, 4]=1
print(a[3, 4])

1.0


### 优秀作业测试
#### Homework 0: Fractal demo热爱者

In [4]:
ti.init(arch=ti.gpu)
n = 320
pixels = ti.var(dt=ti.f32, shape=(n * 2, n))

@ti.func
def complex_power(z, power: ti.i32):
    r = ti.sqrt(z[0]**2 + z[1]**2)
    theta = ti.atan2(z[1], z[0])
    return ti.Vector([r**power * ti.cos(power*theta), r**power * ti.sin(power*theta)])

@ti.kernel
def paint(t: ti.f32, power: ti.i32):
    for i, j in pixels:  # Parallized over all pixels
        freq = 1.0 / power
        c = ti.Vector([0.7885 * ti.cos(freq*t), 0.7885 * ti.sin(freq*t)])
        z = ti.Vector([i / n - 1, j / n - 0.5]) * 2

        iterations = 0
        while z.norm() < 20 and iterations < 50:
            z = complex_power(z, power) + c
            iterations += 1
        pixels[i, j] = 1 - iterations * 0.02

power = eval(input("Power of z -> "))
gui = ti.GUI("Julia Set", res=(n * 2, n))

for i in range(1000000):
    paint(i * 0.03, power)
    gui.set_image(pixels)
    gui.show()

Power of z -> 5


KeyboardInterrupt: 

```flow
st=>start: 开始
op=>operation: My Operation
cond=>condition: Yes or No?
e=>end
st->op->cond
cond(yes)->e
cond(no)->op
&```

### Taichi中文文档学习

#### 基本概念  
##### Using external arrays as Taichi kernel parameters

In [5]:
import taichi as ti
import numpy as np

ti.init()

n = 4
m = 7

val = ti.var(ti.i32, shape=(n, m))

@ti.kernel
def test_numpy(arr: ti.ext_arr()):
    for i in range(n):
        for j in range(m):
            arr[i, j] += i + j

a = np.empty(shape=(n, m), dtype=np.int32)

for i in range(n):
    for j in range(m):
        a[i, j] = i * j
print(a)
test_numpy(a)
print("a afer 1st func", a)
for i in range(n):
    for j in range(m):
        assert a[i, j] == i * j + i + j
print(a)


[[ 0  0  0  0  0  0  0]
 [ 0  1  2  3  4  5  6]
 [ 0  2  4  6  8 10 12]
 [ 0  3  6  9 12 15 18]]
a afer 1st func [[ 0  1  2  3  4  5  6]
 [ 1  3  5  7  9 11 13]
 [ 2  5  8 11 14 17 20]
 [ 3  7 11 15 19 23 27]]
[[ 0  1  2  3  4  5  6]
 [ 1  3  5  7  9 11 13]
 [ 2  5  8 11 14 17 20]
 [ 3  7 11 15 19 23 27]]
